In [1]:
from google.colab import drive
import json
drive.mount('/content/gdrive',force_remount=False)

Mounted at /content/gdrive


In [2]:
!cp /content/gdrive/MyDrive/converted_224x224.tar.gz /content
%cd /content


/content


In [3]:
!tar -xvzf /content/converted_224x224.tar.gz

Streaming output truncated to the last 5000 lines.
converted_224x224/frame_174866.jpg
converted_224x224/frame_106209.jpg
converted_224x224/frame_183385.jpg
converted_224x224/frame_056394.jpg
converted_224x224/frame_065265.jpg
converted_224x224/frame_188455.jpg
converted_224x224/frame_162446.jpg
converted_224x224/frame_032771.jpg
converted_224x224/frame_188246.jpg
converted_224x224/frame_043820.jpg
converted_224x224/frame_035343.jpg
converted_224x224/frame_003077.jpg
converted_224x224/frame_141712.jpg
converted_224x224/frame_000073.jpg
converted_224x224/frame_159991.jpg
converted_224x224/frame_184300.jpg
converted_224x224/frame_129955.jpg
converted_224x224/frame_189617.jpg
converted_224x224/frame_033131.jpg
converted_224x224/frame_083162.jpg
converted_224x224/frame_012496.jpg
converted_224x224/frame_012634.jpg
converted_224x224/frame_004062.jpg
converted_224x224/frame_036170.jpg
converted_224x224/frame_147494.jpg
converted_224x224/frame_163792.jpg
converted_224x224/frame_052241.jpg
conv

In [4]:
!ls /content/converted_224x224/

Streaming output truncated to the last 5000 lines.
frame_041876.jpg  frame_094602.jpg  frame_144284.jpg  frame_186186.jpg
frame_041878.jpg  frame_094603.jpg  frame_144285.jpg  frame_186187.jpg
frame_041879.jpg  frame_094604.jpg  frame_144286.jpg  frame_186188.jpg
frame_041881.jpg  frame_094605.jpg  frame_144287.jpg  frame_186189.jpg
frame_041882.jpg  frame_094606.jpg  frame_144288.jpg  frame_186190.jpg
frame_041883.jpg  frame_094607.jpg  frame_144291.jpg  frame_186191.jpg
frame_041886.jpg  frame_094608.jpg  frame_144295.jpg  frame_186192.jpg
frame_041887.jpg  frame_094612.jpg  frame_144297.jpg  frame_186193.jpg
frame_041889.jpg  frame_094613.jpg  frame_144298.jpg  frame_186195.jpg
frame_041890.jpg  frame_094614.jpg  frame_144299.jpg  frame_186196.jpg
frame_041891.jpg  frame_094617.jpg  frame_144301.jpg  frame_186197.jpg
frame_041892.jpg  frame_094619.jpg  frame_144302.jpg  frame_186198.jpg
frame_041894.jpg  frame_094621.jpg  frame_144304.jpg  frame_186199.jpg
frame_041896.jpg  frame_09

In [7]:
import os,inspect
import sys
import time

import numpy as np
import pandas as pd

import pdb
import gc
from tqdm.notebook import trange, tqdm
from PIL import Image

import torch
import torchvision 
import torch.nn as nn

#from torch.cuda.amp import GradScaler
#from torch.cuda.amp import autocast

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

num_workers = 4 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers),  " system version = ", sys.version)

Cuda =  True  with num_workers =  4  system version =  3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


# 1. PairWise DataLoader


# 2. Data Loading Example


In [8]:
np.random.seed(0)
df = pd.read_csv("/content/gdrive/MyDrive/pair_out.csv")
print(df.shape[0])
df['split'] = np.random.randn(df.shape[0], 1)
df = df.iloc[:, 1:]
train_msk = df['split'] <= 0.7
val_msk = (df['split'] > 0.7) & (df['split'] <= 0.85)
test_msk = df['split'] > 0.85
train_data = df[train_msk]
val_data = df[val_msk]
test_data = df[test_msk]


119593


In [9]:
train_data = train_data.drop('split', 1).reset_index().drop('index',1)
val_data = val_data.drop('split',1).reset_index().drop('index',1)
test_data = test_data.drop('split',1).reset_index().drop('index',1)

In [10]:
# Define dataset class


class MyDataSet(Dataset):

    # load the dataset
    def __init__(self, data, **kwargs):
        self.X = data["frames"]
        self.Y = data["label"]

    # get number of items/rows in dataset
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, index):
        x, y = self.X[index], self.Y[index]
        for c in ["[","]",",","'"]:
          x = x.replace(c, "")
          y = y.replace(c,"")
        x = x.split(" ")
        y = y.split(" ")
        y = list(map(lambda x:int(x),y))
        y = torch.FloatTensor(y)
        images = []
        for img_file in x:
            img = Image.open(cur_dir + img_file)
            img = torchvision.transforms.ToTensor()(img)
            img = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])(img)
            images.append(img)
        x = torch.stack(images)
        return x, y

In [11]:
cur_dir = 'converted_224x224/'

In [12]:
class TestDataSet(Dataset):

    # load the dataset
    def __init__(self, data, **kwargs):
        self.X = data["frames"]

    # get number of items/rows in dataset
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        x = self.X[index]
        for c in ["[","]",",","'"]:
          x = x.replace(c, "")
        x = x.split(" ")
        images = []
        for img_file in x:
            img = Image.open(cur_dir + img_file)
            img = torchvision.transforms.ToTensor()(img)
            img = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])(img)
            images.append(img)
        x = torch.stack(images)
        return x

In [13]:
batch_size = 16

# training data
train_set = MyDataSet(train_data)
train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size, num_workers=8)

# validation data
val_set = MyDataSet(val_data)
val_loader = DataLoader(val_set, shuffle=False, batch_size=batch_size, num_workers=8)
# test data
test_set = TestDataSet(test_data)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size, num_workers=8)

In [14]:
# Model 

#These libraries help to interact with the operating system and the runtime environment respectively
import os
import sys

#Model/Training related libraries
import numpy as np
import torch
import torch.nn as nn
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import time
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms, datasets

#Dataloader libraries
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [15]:
class CNN_Feature_Extractor(torch.nn.Module):
    def __init__(self, feat_emb, pretrained=True):
        super().__init__()
        self.efficientnet = torchvision.models.efficientnet_b0(pretrained=pretrained)
        self.linear = nn.Linear(1000, feat_emb)

    def forward(self, x):
        x = self.efficientnet(x)
        x = self.linear(x)
        return x

In [16]:
class ClassificationNetwork(nn.Module):
    def __init__(self, in_features, num_classes, feat_dim=512):
        super().__init__()
        
        self.cnn = CNN_Feature_Extractor(512, pretrained=False)

        self.linear_emb = nn.Linear(512, feat_dim)
        self.relu = nn.ReLU(inplace=True)
        # output of CNN is b*5, 512 
        # before put into linear, has to reshape to x
        self.linear_output_1 = nn.Linear(1024,512)
        self.relu_output = nn.ReLU(inplace=True)
        self.dropout_output = nn.Dropout(p=0.4)
        self.linear_output_2 = nn.Linear(512,num_classes)
        self.sigm = nn.Sigmoid()


    def forward(self, x, return_embedding=False):
        embedding = self.cnn(x)
        embedding_out = self.linear_emb(embedding)
        embedding_out = self.relu(embedding_out)

        # output of CNN is b*5, 512 
        # before put into linear, has to reshape to x
        embedding.requires_grad_(True)
        embedding_reshape = torch.reshape(embedding, (-1,5,512))
        # embedding shape is b * 5 * 512
        # slicing and stack
        final_embeddings = []
        B,I,_ = embedding_reshape.shape
        for b in range(B):
          batch = embedding_reshape[b,:,:]
          batch_pair_embeddings = []
          for i in range(I):
            for j in range(i+1, I):
              pair_embedding_1 = batch[i,:]
              pair_embedding_2 = batch[j,:]
              pairs = torch.cat((pair_embedding_1,pair_embedding_2),dim=0)
              batch_pair_embeddings.append(pairs)
          paired_embeddings = torch.stack(batch_pair_embeddings)
          final_embeddings.append(paired_embeddings)
        # batch * 10 , 1024
        paris_embdeddings = torch.stack(final_embeddings).reshape(-1,1024) 
        output = self.linear_output_1(paris_embdeddings)
        output = self.relu_output(output)
        output = self.dropout_output(output)
        # output dimension is b*10, 1
        output = self.linear_output_2(output)
        output = self.sigm(output)

        if return_embedding:
            return embedding_out,output
        else:
            return output 

In [17]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
 
    def forward(self, out, targets):
      # out shape (b*10, 1)
      # target shape batch * 10
      targets = targets.reshape(-1,1)
      batch, pairs = targets.shape
      loss = torch.nn.BCELoss()(out, targets)
      return loss    

In [18]:
# Train the model

def train_model(train_loader,train_dataset,model,criterion, optimizer):
    training_loss = 0
    accuracy = 0.0
    # Set model in 'Training mode'
    model.train()
    start_time = time.time()
    with tqdm(train_loader) as t:
      for i, (x, y) in enumerate(train_loader):
        t.set_description("Training")
        # clear the gradients
        optimizer.zero_grad()
        x.requires_grad_(True)
        # all data & model on same device
        x = torch.reshape(x, (-1,3, 224, 224))
        x = x.to(device)
        y = y.to(device)
        # compute the model output
        out = model(x)
        # y shape: batch * 10
        # calculate loss
        loss = criterion(out, y)
        # call scalar loss
        # Backward pass
        loss.backward()
        # Update model weights 
        optimizer.step()
       
        #caculate accuracy
        p = torch.reshape(out, (-1, 10) )
        predict = (p > 0.5) * (torch.ones(p.shape).to(device))
        accuracy += (predict == y).sum().item()
        
        training_loss += loss.item()

    end_time = time.time()
    acc = accuracy/(len(train_dataset)*10)
    training_loss /= len(train_loader)
    print('Training Loss: ', training_loss, 'Time: ',end_time - start_time, 's')
    print("Training accuracy: ", acc)
    torch.cuda.empty_cache()
    del x
    del y
    del loss
    return training_loss,acc

In [19]:
# Evaluate the model

def evaluate_model(val_loader, val_dataset, model,criterion):
  with torch.no_grad():
    model.eval()
    val_loss = 0.0
    accuracy = 0
    # Set model in validation mode
    
    with tqdm(val_loader) as t:
      for i, (x, y) in enumerate(val_loader):
        t.set_description("Evaluating")
         # all data & model on same device
        x = torch.reshape(x, (-1,3, 224, 224))
        x = x.to(device)
        y = y.to(device)
        # evaluate the model on the validation set
        out = model(x)  
        # Calculate validation loss
        # reshape the result from (b*10, 1) to b * 10
        # calculate loss
        loss = criterion(out, y)
  
         #caculate accuracy
        p = torch.reshape(out, (-1, 10) )
        predict = (p > 0.5) * (torch.ones(p.shape).to(device))
        accuracy += (predict == y).sum().item()
        
        val_loss += loss.item()
    
    # Calculate validation accuracy
    acc = accuracy/(len(val_dataset)*10)
    # calculate val loss
    val_loss /= len(val_loader)
    return val_loss,acc



In [20]:
numEpochs = 80
in_features = 3 # RGB channels
learningRate = 1e-1
weightDecay = 1e-4
num_classes = 1 # only 0 or 1


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ClassificationNetwork(in_features, num_classes)
model = model.to(device)
print(model)
# Define customized Loss function
criterion = CustomLoss()

# Adam
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3,threshold=0.01)

ClassificationNetwork(
  (cnn): CNN_Feature_Extractor(
    (efficientnet): EfficientNet(
      (features): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): Sequential(
          (0): MBConv(
            (block): Sequential(
              (0): ConvNormActivation(
                (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): SiLU(inplace=True)
              )
              (1): SqueezeExcitation(
                (avgpool): AdaptiveAvgPool2d(output_size=1)
                (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
                (fc2): Conv2d(8, 32, kernel_size=(1, 1), str

In [ ]:
base_acc = 0.0

# Define number of epochs
for epoch in range(numEpochs):
    
    # Train
    training_loss, train_acc= train_model(train_loader,train_set,model,criterion, optimizer)
    
    # Validation for validation dataset
    val_loss,val_acc = evaluate_model(val_loader,val_set,model,criterion)

    # scheduler
    scheduler.step(val_acc)
    print("Epoch: ",epoch+1,", current lr:", optimizer.state_dict()['param_groups'][0]['lr'])

    if val_acc > base_acc:
      model_save_name = 'shot_detection_EfficientNet_Pairwise.pt'
      path = F"/content/gdrive/My Drive/{model_save_name}"
      print("Epoch: "+str(epoch+1)+",saved model parameter") 
      torch.save(model.state_dict(), path)
      base_acc = val_acc
    
    # Print log of training & validate accuracy
    print("Epoch: "+str(epoch+1)+", Training accuracy: "+str(train_acc*100)+"%, Validation accuracy:"+str(val_acc*100)+"%, Training loss: "+str(training_loss)+", Validation loss:"+str( val_loss))

Training:   0%|          | 0/5668 [34:02<?, ?it/s]


Training Loss:  49.945932806747514 Time:  2042.0178129673004 s
Training accuracy:  0.5003330576233802


Evaluating:   0%|          | 0/331 [01:01<?, ?it/s]


Epoch:  1 , current lr: 0.1
Epoch: 1,saved model parameter
Epoch: 1, Training accuracy: 50.03330576233802%, Validation accuracy:50.797128825085004%, Training loss: 49.945932806747514, Validation loss:49.20128398791541


Training:   0%|          | 0/5668 [35:42<?, ?it/s]


...

Training:   0%|          | 0/5668 [35:24<?, ?it/s]


Training Loss:  50.011100329558616 Time:  2124.752760410309 s
Training accuracy:  0.49989853873724843


Evaluating:   0%|          | 0/331 [01:08<?, ?it/s]


Epoch:  8 , current lr: 0.05
Epoch: 8, Training accuracy: 49.98985387372484%, Validation accuracy:50.797128825085004%, Training loss: 50.011100329558616, Validation loss:49.20128398791541


Training:   0%|          | 0/5668 [12:21<?, ?it/s]

In [ ]:
torch.cuda.empty_cache()
with torch.no_grad():
  model.eval()
  accuracy = 0
  with tqdm(test_loader) as t:
    for i, (x, y) in enumerate(test_loader):
      t.set_description("Testing")
      # all data & model on same device
      x = torch.reshape(x, (-1,3, 224, 224))
      x = x.to(device)
      y = y.to(device)
      # evaluate the model on the validation set
      out = model(x)  
      #caculate accuracy
      p = torch.reshape(out, (-1, 10) )
      predict = (p > 0.5) * (torch.ones(p.shape).to(device))
      accuracy += (predict == y).sum().item()

    test_acc = accuracy/ (len(test_set)*10)
    print(test_acc)